In [ ]:
import numpy as np
from PIL import Image
from glob import glob # manipulação de lista de nomes de arquivos
from skimage.measure import regionprops # propriedades de cada segmento
import matplotlib.pyplot as plt

import seaborn as sns # gráficos
import pandas as pd # manipulação de dataframes (sheets)

In [ ]:
# Célula com funções para investigar se um segmento é misto, obter HoR e obter a classe majoritária de um segmento

# 0 - Fundo
# 1 - Desmatamento recente
# 2 - Floresta
# 3 - Ñ analisado

def is_mixed(segment):
    # flattening segment
    segment = segment.flatten()


    NFP = np.count_nonzero(segment == 2) # se floresta
    NP = np.count_nonzero(segment) # desconsiderando o fundo (np.zeros)
    NNP = NP - NFP # Calcula os pixels de não-fundo

    if NFP != 0 and NNP != 0: # se em um segmento tiver pixels de desmata e floresta, ele é misto
        return True
    
    return False
    

def get_hor(segment):
    
    # flattening segment (transformando a matriz 'segment' em um array unidimensional)
    segment = segment.flatten()

    # Number Foreground Pixels: conta o número de pixels no vetor segment que são diferentes de zero. Especificamente, os que têm o valor 2
    NFP = np.count_nonzero(segment == 2)

    # conta o número total de pixels no vetor 'segment' que são diferentes de zero. Isso representa o número total de pixels
    NP = np.count_nonzero(segment) # desconsiderando o fundo
    
    # Número de pixels de não-fundo: subtração da quantidade de pixels total de não fundo pela quantidade de pixels iguais a 170
    NNP = NP - NFP

    HoR = max([NFP, NNP]) / NP

    return HoR


def get_major_class(segment):
    if np.argmax(np.bincount(segment.flatten())) == 2:
        return "forest"
    elif np.argmax(np.bincount(segment.flatten())) == 1:
        return "non forest"
    elif np.argmax(np.bincount(segment.flatten())) == 3:
        return "not analyzed"
    else:
        return np.argmax(np.bincount(segment.flatten()))

In [ ]:
# Construir as listas de arquivos a processar
base = 'data/'
slic_path = base + 'saved_slic/'
file_list = [
    'ae03_sentinel_rgb.npy',
    'AREA3_c_10000_hls_mask_corrigido.npy', 
    'AREA3_c_10000_rw_mask_corrigido.npy', 
    'AREA3_c_10000_snll_mask_corrigido.npy', 
    'AREA3_c_10000_wishart_mask_corrigido.npy']
